In [ ]:
# Download and unpack data, then delete temporary files:
# !! bash ../download_data.sh

# Heirarchical Attention Network for text classification

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

from src.notebooks_utils import display_predict
from src.dataset import get_test_dataset, collate_docs, ImdbReviewsDataset
from src.model import HAN
from src.const import LOG_DIR_CATALYST

## Load pretrained model and data

In [ ]:
model = HAN.from_imbd_ckpt(path_to_ckpt=LOG_DIR_CATALYST / 'checkpoints' / 'best.pth')

dataset = get_test_dataset()
itow = dict(zip(dataset.vocab.values(), dataset.vocab.keys()))

## Display predict for reviews from test

In [ ]:
from random import randint

idx = randint(1, len(dataset))
batch = collate_docs([dataset[idx]])
display_predict(model=model, batch=batch, itow=itow)
    
print('Raw review:')
print(open(dataset._paths[idx], 'r').read())

## Display predict on your own review

In [ ]:
text_str = '''

I really like films like that! I discover new talents for myself. 
The sound was great and picture as well. So, I think I will see it again.

'''


text, snt_max_len, txt_len  = dataset.tokenize_plane_text(text_str)

batch = collate_docs([{'txt': text, 'snt_len': snt_max_len,
                       'txt_len': txt_len, 'label': -1}])
batch['targets'] = None

display_predict(model=model, batch=batch, itow=itow)